In [1]:
##################################
# Calculate HGM
##################################

In [2]:
##################################
# vflood
##################################

In [3]:
import os
import rasterio
import datetime
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rasterio.mask import mask
from rasterio.plot import show
from shapely.geometry import shape, box
from functools import reduce
from pathlib import Path

In [4]:
#savefile = 'exampleprojectsAIO'
savefile = 'yazoo_mitigation_potential_sites_output'
workspace = r'D:\gis\projects\YazooHGM\exampleprojects'
floodfiles = ['alt2_2year.tif','alt2_5year.tif']

aoi = os.path.join(workspace,'yazoo_study_area.shp')
foi = os.path.join(workspace, 'examples.shp')
#foi = os.path.join(workspace, 'yazoo_mitigation_potential_sites.shp')
pv = os.path.join(workspace, 'Yazoo_PNV.shp')
floodar = [os.path.join(workspace, fl) for fl in floodfiles]

ac_ = 0.000247105
ha_ = 0.0001

In [5]:
aoi = gpd.read_file(aoi).to_crs(5070)
foi = gpd.read_file(foi).to_crs(5070)
foi['uid'] = foi.index.map(hash)
foi['origacres'] = foi.area * ac_
pv = gpd.read_file(pv).to_crs(5070)

#foi = foi[foi['site_id']==3]

In [6]:
if 'Name' not in foi.columns:
    foi['Name'] = foi['uid']

In [7]:
foi = foi[['Name', 'uid', 'origacres', 'geometry']]

In [8]:
cogin = 'https://giscog.blob.core.windows.net/newcontainer/nlcd2019_cog.tif'
cog = rasterio.open(cogin)
aoicrs = aoi.to_crs(cog.crs)
nlcd, nlcd_transform =rasterio.mask.mask(cog, aoicrs.dissolve().geometry, crop=True)
ag = nlcd.copy()
ag[ag>82]=0
ag[ag<81]=0
ag[ag==81]=1
ag[ag==82]=1
shapes = rasterio.features.shapes(ag[0], transform=nlcd_transform, mask=ag[0] == 1)
ag = gpd.GeoDataFrame({'geometry': [shape(geom) for geom, value in shapes]})
ag = ag.set_geometry('geometry')
ag = ag.set_crs(5070)
woody = nlcd.copy()
woody[(woody>90) & (woody <90)]=0
woody[woody==90]=1
shapes = rasterio.features.shapes(woody[0], transform=nlcd_transform, mask=woody[0] == 1)
woody = gpd.GeoDataFrame({'geometry': [shape(geom) for geom, value in shapes]})
woody = woody.set_geometry('geometry')
woody = woody.set_crs(5070)

In [9]:
clippedwoody = gpd.clip(woody, aoi)
foi = gpd.clip(foi, ag)
foi['uid'] = foi.index.map(hash)
foi['acres'] = foi.area * ac_

In [29]:
storeflood = []
for fld in floodar:
    flood = rasterio.open(fld)
    flood, flood_transform =rasterio.mask.mask(flood, foi.dissolve().geometry, crop=True)
    shapes = rasterio.features.shapes(flood[0], transform=flood_transform, mask=flood[0]==1)
    geoms = []
    values = []

    for geom,value in shapes:
        geoms.append(shape(geom))
        values.append(value)
    flood_vector = gpd.GeoDataFrame({'geometry': geoms, 'value': values})
    flood_vector = flood_vector.set_crs(5070)
    foipv = gpd.overlay(foi, pv, how='intersection')
    foipv['veg']=foipv['HGM'].str.split('-', expand=True)[0]
    foipv = foipv.dissolve(by=['uid', 'veg'])
    foiflood = foipv
    foiflood['vegtype'] = np.select(
        [
            foiflood['HGM'].str.contains('F-'), 
            foiflood['HGM'].str.contains('RB'),
            foiflood['HGM'].str.contains('RO'),
            foiflood['HGM'].str.contains('D-3'),
            foiflood['HGM'].str.contains('D-4'),
            foiflood['HGM'].str.contains('D-1'),
            foiflood['HGM'].str.contains('D-2')
        ], 
        ['flats', 'backwater', 'overbank','isolated', 'isolated', 'connected', 'connected'], 
        default='Unknown'
    )
    #
    foiflood['vegacres'] = foiflood.area * ac_
    foiflood = gpd.overlay(foiflood, flood_vector, how='intersection')
    foiflood = foiflood.dissolve(by=['Name', 'vegtype', 'vegacres'], aggfunc={'value':'mean'})
    foiflood = foiflood.reset_index()
    foiflood = foiflood[foiflood['vegtype']!='Unknown']
    foiflood = foiflood.rename(columns={'value':'vfreq'})
    if fld == floodar[0]:
        foiflood['vfreq'] = 2
    else:
        foiflood['vfreq'] = 5
    foiflood['floodacres'] = foiflood.area * ac_
    storeflood.append(foiflood)

In [30]:
storeflood

[            Name    vegtype  vegacres  \
 0  HGM example 1  backwater     881.6   
 1  HGM example 1      flats      96.7   
 2  HGM example 1   isolated       6.0   
 4  HGM example 2  backwater     763.8   
 5  HGM example 2   isolated     102.7   
 6  HGM example 3  backwater     350.8   
 7  HGM example 4  backwater      35.2   
 8  HGM example 5  backwater   1,021.3   
 9  HGM example 5      flats     441.9   
 
                                             geometry  vfreq  floodacres  
 0  MULTIPOLYGON Z (((480765.643 1078634.281 0.000...      2       690.4  
 1  POLYGON Z ((480221.647 1078235.383 0.000, 4802...      2         0.0  
 2  POLYGON Z ((480656.799 1077857.039 0.000, 4806...      2         6.0  
 4  MULTIPOLYGON Z (((483106.512 1070654.601 0.000...      2       281.2  
 5  MULTIPOLYGON Z (((483883.754 1069354.142 0.000...      2        16.1  
 6  MULTIPOLYGON Z (((483594.193 1061656.887 0.000...      2        16.1  
 7  MULTIPOLYGON Z (((468765.000 1061568.495 0.000...

In [31]:
##################################
# vtract, vcore
##################################

In [32]:
# vcore is ag area with -100m buffer to calculate ag area core.  Divide by ag area to get percent core.
foi['vcore'] = ((foi['geometry'].buffer(-100).area * ac_)/foi['acres'])*100
hold = gpd.GeoDataFrame()
for ft in foi.iterfeatures():
    ws = gpd.GeoDataFrame.from_features([ft],crs="EPSG:5070")
    wsbuffer = ws.copy()
    wsbuffer['geometry'] = ws['geometry'].buffer(10)
    selectwoody = gpd.sjoin(clippedwoody[['geometry']],wsbuffer, how='inner',predicate='intersects')
    ws = gpd.overlay(ws, selectwoody, how='union')
    ws = ws.dissolve()
    hold = pd.concat([hold, ws])
hold['vtract'] = hold.area * ha_

hold = hold.drop(columns=['index_right'])
hold = hold.rename(columns={'Name_1': 'Name', 'uid_1': 'uid', 'acres_1': 'acres'}).drop(columns=['acres_2', 'uid_2'])

bigbuffer = foi.copy()
bigbuffer['geometry'] = bigbuffer.buffer(500).boundary
bigbuffer['perim'] = bigbuffer.length
bigbuffer = bigbuffer.overlay(clippedwoody, how='intersection')
bigbuffer = bigbuffer.dissolve(by='uid', aggfunc={'perim':'first'})
bigbuffer['connectperim'] = bigbuffer.length
bigbuffer['vconnect'] = (bigbuffer['connectperim']/bigbuffer['perim'])*100
bigbuffer = bigbuffer.reset_index()
hold = hold.merge(bigbuffer[['uid', 'vconnect']], on='uid')
outfoi = foi.merge(hold[['uid', 'vtract', 'vconnect']], on='uid')


D:\mambaforge\envs\newduckdb\lib\site-packages\geopandas\geodataframe.py:1815: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  merged_geom = block.unary_union
C:\Users\mmitchell.DUCKS\AppData\Local\Temp\ipykernel_29852\616919326.py:9: UserWarning: `keep_geom_type=True` in overlay resulted in 14 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  ws = gpd.overlay(ws, selectwoody, how='union')
D:\mambaforge\envs\newduckdb\lib\site-packages\geopandas\geodataframe.py:1815: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  merged_geom = block.unary_union
C:\Users\mmitchell.DUCKS\AppData\Local\Temp\ipykernel_29852\616919326.py:9: UserWarning: `keep_geom_type=True` in overlay resulted in 18 dropped geometries of different geometry types than 

In [34]:
'''
Flats - flats
Riverine backwater - backwater
Riverine overbank - overbank
Isolated depressions - isolated
Connected depressions - connected
'''
# if vtract >= 3000: return 1 else return 0.0003*vtract
def calc_vtract(vtract: float, subclass: str = '') -> float:
    if vtract >= 3000:
        return 1
    else:
        return 0.0003*vtract            

# if vcore >= 40: return 1 else return 0.025*vcore
def calc_vcore(vcore: float, subclass: str = '') -> float:
    if vcore >= 40:
        return 1
    else:
        return 0.025*vcore

# if vconnect >= 40 return 1 else return 0.025*vconnect
def calc_vconnect(vconnect: float, subclass: str = '') -> float:
    if vconnect >= 40:
        return 1
    else:
        return 0.025*vconnect    
    
# if vfreq > 2: return 1.3333+(-0.16667*vfreq) elif vfreq <= 2: return 1
def calc_vfreq(vfreq: float, subclass: str) -> float:
    match subclass:
        case 'flats':
            return 0
        case _:
            if vfreq > 2:
                return 1.3333+(-0.16667*vfreq)
            else:
                return 1

# if vpond >= 70 return 2.16667+(-0.01667*vpond) elif vpond >=40 return 1 else return 0.025*vpond
def calc_vpond(vpond: float, subclass: str) -> float:
    match subclass:
        case 'flats':
            if vpond >= 70:
                return 2.16667+(-0.01667*vpond)
            elif vpond >= 40:
                return 1
            else:
                return 0.025*vpond
        case 'backwater':
            if vpond >= 70:
                return 2.16667+(-0.01667*vpond)
            elif vpond >= 40:
                return 1
            else:
                return 0.025*vpond        
        case 'overbank': #IF(C11>=70,0.5,IF(C11>40,1.666667+(-0.01667*C11),IF(C11>=0,1)))
            if vpond >= 70:
                return 0.5
            elif vpond >= 40:
                return 1.666667+(-0.01667*vpond)
            elif vpond >=0:
                return 1
            else:
                return 0
        case _:
            return 0

            
# if vsoil <=100 return 1-(0.01*vsoil) else 0
def calc_vsoil(vsoil: float, subclass: str = '') -> float:
    if vsoil <= 100:
        return 1-(0.01*vsoil)
    else:
        return 0
    
# if vcec <=100 return 1-(0.01*vcec) else return 0
def calc_vcec(vcec: float, subclass: str) -> float:
    match subclass:
        case 'isolated':
            return 0
        case _:
            if vcec <=100:
                return 1-(0.01*vcec) 
            else:
                return 0
        
# if vtba >=30 return 1 else return 0.03333*vtba
def calc_vtba(vtba: float, subclass: str) -> float:
    match subclass:
        case 'flats':
            if vtba >=30:
                return 1
            else:
                return 0.03333*vtba
        case 'backwater':
            if vtba >=20:
                return 1
            else:
                return 0.05*vtba
        case 'overbank':
            if vtba >= 25:
                return 1
            else:
                return 0.04*vtba
        case 'isolated':
            if vtba >=30:
                return 1
            else:
                return 0.03333*vtba
        case 'connected':
            if vtba >=30:
                return 1
            else:
                return 0.03333*vtba            
    
# if vtden >=800 return 0.5 elif vtden > 500 return 2.25+(-0.0025*vtden) elif vtden >=250 return 1 else return 0.004*vtden
def calc_vtden(vtden: float, subclass: str) -> float:
    match subclass:
        case 'flats':    
            if vtden >= 800:
                return 0.5
            elif vtden > 500:
                return 2.25+(-0.0025*vtden)
            elif vtden >=250:
                return 1
            else:
                return 0.004*vtden
        case 'backwater': # IF(C15>=800,0.5,IF(C15>600,2.5+(-0.0025*C15),IF(C15>=250,1,IF(C15>=0,0+(0.004*C15))))
            if vtden >= 800:
                return 0.5
            elif vtden > 600:
                return 2.25+(-0.0025*vtden)
            elif vtden >=250:
                return 1
            else:
                return 0.004*vtden          
        case 'overbank': # IF(C15>=800,0.5,IF(C15>600,2.5+(-0.0025*C15),IF(C15>=300,1,IF(C15>=0,0+(0.003333*C15))))
            if vtden >= 800:
                return 0.5
            elif vtden > 600:
                return 2.25+(-0.0025*vtden)
            elif vtden >=300:
                return 1
            else:
                return 0.003333*vtden       
        case 'isolated': # IF(C15>=900,0.5,IF(C15>800,5+(-0.005*C15),IF(C15>=250,1,IF(C15>=0,0+(0.004*C15))))
            if vtden >= 900:
                return 0.5
            elif vtden > 800:
                return 5+(-0.005*vtden)
            elif vtden >=250:
                return 1
            else:
                return 0.004*vtden        
        case 'connected': # IF(C15>=800,0.5,IF(C15>700,4.5+(-0.005*C15),IF(C15>=250,1,IF(C15>=0,0+(0.004*C15))))
            if vtden >= 800:
                return 0.5
            elif vtden > 700:
                return 4.5+(-0.005*vtden)
            elif vtden >=250:
                return 1
            else:
                return 0.004*vtden          
    
# if vsnag>=100 return 0.5 elif vsnag >= 80 return 3+(-0.025*vsnag) elif vsnag >=50 return 1 else return 0.02*vsnag    
def calc_vsnag(vsnag: float, subclass: str) -> float:
    match subclass:
        case 'flats': # IF(C16>=100,0.5,IF(C16>=80,3+(-0.025*C16),IF(C16>=50,1,IF(C16>=0,0+(0.02*C16))))
            if vsnag>=100:
                return 0.5
            elif vsnag >= 80:
                return 3+(-0.025*vsnag)
            elif vsnag >=50:
                return 1
            else:
                return 0.02*vsnag
        case 'backwater': # IF(C16>=100,0.5,IF(C16>=60,1.75+(-0.0125*C16),IF(C16>=25,1,IF(C16>=0,0+(0.04*C16))))
            if vsnag>=100:
                return 0.5
            elif vsnag >= 60:
                return 1.75+(-0.0125*vsnag)
            elif vsnag >=25:
                return 1
            else:
                return 0.04*vsnag
        case 'overbank': # IF(C16>=100,0.5,IF(C16>=50,1.5+(-0.01*C16),IF(C16>=15,1,IF(C16>=0,0.5+(0.033333*C16))))
            if vsnag>=100:
                return 0.5
            elif vsnag >= 50:
                return 1.5+(-0.01*vsnag)
            elif vsnag >=15:
                return 1
            else:
                return 0.5+(0.033333*vsnag)
        case 'isolated': # IF(C16>=100,0.5,IF(C16>=70,2.166667+(-0.01667*C16),IF(C16>=30,1,IF(C16>=0,0+(0.03333*C16))))
            if vsnag>=100:
                return 0.5
            elif vsnag >= 70:
                return 2.166667+(-0.01667*vsnag)
            elif vsnag >=30:
                return 1
            else:
                return 0.033333*vsnag
        case 'connected': # IF(C16>=120,0.5,IF(C16>=60,1.5+(-0.00833*C16),IF(C16>=20,1,IF(C16>=0,0+(0.05*C16))))
            if vsnag>=120:
                return 0.5
            elif vsnag >= 60:
                return 1.5+(-0.00833*vsnag)
            elif vsnag >=20:
                return 1
            else:
                return 0.05*vsnag               
            
# if vtcomp >=0 return 0.01*vtcomp else 0
def calc_vtcomp(vtcomp: float, subclass: str) -> float:
    if vtcomp >= 0:
        return 0.01*vtcomp
    else:
        return 0

# if vcomp >= 0 return 0.01*vcomp else return 0
def calc_vcomp(vcomp: float, subclass: str) -> float:
    if vcomp >= 0:
        return 0.01*vcomp
    else:
        return 0

# if vwd >= 130 return 0.5 elif vwd > 90 return 2.125+(-0.0125*vwd) elif vwd >30 return 1 else return 0.033333*vwd
def calc_vwd(vwd: float, subclass: str) -> float:
    match subclass:
        case 'flats':    
            if vwd >= 130:
                return 0.5
            elif vwd > 90:
                return 2.125+(-0.0125*vwd)
            elif vwd >30:
                return 1
            else:
                return 0.033333*vwd
        case 'backwater': # IF(C19>=100,0.5,IF(C19>80,3+(-0.025*C19),IF(C19>=25,1,IF(C19>=0,0+(0.04*C19))))
            if vwd >= 100:
                return 0.5
            elif vwd > 80:
                return 3+(-0.025*vwd)
            elif vwd >25:
                return 1
            else:
                return 0.04*vwd
        case 'overbank': # IF(C19>=100,0.5,IF(C19>80,3+(-0.025*C19),IF(C19>=25,1,IF(C19>=0,0+(0.04*C19))))
            if vwd >= 100:
                return 0.5
            elif vwd > 80:
                return 3+(-0.025*vwd)
            elif vwd >25:
                return 1
            else:
                return 0.04*vwd
        case _:
            return 0

# if vlog >= 100 return 0.5 elif vlog>75, return 2.5+(-0.02*vlog) elif vlog >= 25 return 1 else return 0.04*vlog
def calc_vlog(vlog: float, subclass: str) -> float:
    match subclass:
        case 'flats': # IF(C20>=100,0.5,IF(C20>=75,2.5+(-0.02*C20),IF(C20>=25,1,IF(C20>=0,0+(0.04*C20))))
            if vlog >= 100:
                return 0.5
            elif vlog > 75:
                return 2.5+(-0.02*vlog)
            elif vlog > 25:
                return 1
            else:
                return 0.04*vlog
        case 'backwater': # IF(C20>=100,0.5,IF(C20>=75,2.5+(-0.02*C20),IF(C20>=25,1,IF(C20>=0,0+(0.04*C20))))
            if vlog >= 100:
                return 0.5
            elif vlog > 75:
                return 2.5+(-0.02*vlog)
            elif vlog > 25:
                return 1
            else:
                return 0.04*vlog
        case 'overbank': # IF(C20>=100,0.5,IF(C20>=75,2.5+(-0.02*C20),IF(C20>=25,1,IF(C20>=0,0+(0.04*C20))))
            if vlog >= 100:
                return 0.5
            elif vlog > 75:
                return 2.5+(-0.02*vlog)
            elif vlog > 25:
                return 1
            else:
                return 0.04*vlog
        case _:
            return 0

# if vssd >= 6000 return 0.5 elif vssd >=4000 return 2+(-0.00025*vssd)  else if vssd >= 1500 return 1 else return 0.000667*vssd
def calc_vssd(vssd: float, subclass: str) -> float:
    match subclass:
        case 'flats': # IF(C21>=6000,0.5,IF(C21>=4000,2+(-0.00025*C21),IF(C21>=1500,1,IF(C21>=0,0+(0.000667*C21))))
            if vssd >= 6000:
                return 0.5
            elif vssd >= 4000:
                return 2+(-0.00025*vssd)
            elif vssd >= 1500:
                return 1
            else:
                return 0.000667*vssd
        case 'backwater': # IF(C21>=7500,0.5,IF(C21>=4000,1.571429+(-0.00014*C21),IF(C21>=1500,1,IF(C21>=0,0+(0.000667*C21))))
            if vssd >= 7500:
                return 0.5
            elif vssd >= 4000:
                return 1.571429+(-0.00014*vssd)
            elif vssd >= 1500:
                return 1
            else:
                return 0.000667*vssd
        case 'overbank': # IF(C21>=6000,0.5,IF(C21>=2500,1.357143+(-0.00014*C21),IF(C21>=500,1,IF(C21>=0,0+(0.002*C21))))
            if vssd >= 6000:
                return 0.5
            elif vssd >= 2500:
                return 1.357143+(-0.00014*vssd)
            elif vssd >= 500:
                return 1
            else:
                return 0.002*vssd
        case 'isolated': # IF(C21>=8000,0.5,IF(C21>=4000,1.5+(-0.00013*C21),IF(C21>=1000,1,IF(C21>=0,0+(0.001*C21))))
            if vssd >= 8000:
                return 0.5
            elif vssd >= 4000:
                return 1.5+(-0.00013*vssd)
            elif vssd >= 1000:
                return 1
            else:
                return 0.001*vssd
        case 'connected': # IF(C21>=7000,0.5,IF(C21>=5000,2.25+(-0.00025*C21),IF(C21>=2000,1,IF(C21>=0,0+(0.0005*C21))))
            if vssd >= 7000:
                return 0.5
            elif vssd >= 5000:
                return 2.25+(-0.00025*vssd)
            elif vssd >= 2000:
                return 1
            else:
                return 0.0005*vssd                                
    
# vgvc >80 return 0.05 elif vgvc>=50 return 1.83333+(-0.01667*vgvc) elif vgvc>=10 return 1 else return 0.1*vgvc
def calc_vgvc(vgvc: float, subclass: str) -> float:
    match subclass:
        case 'flats': # IF(C22>=80,0.5,IF(C22>=50,1.83333+(-0.01667*C22),IF(C22>=10,1,IF(C22>=0,0+(0.1*C22))))
            if vgvc >= 80:
                return 0.5
            elif vgvc >= 50:
                return 1.83333+(-0.01667*vgvc)
            elif vgvc >= 10:
                return 1
            else:
                return 0.1*vgvc
        case 'backwater': # IF(C22>=80,0.5,IF(C22>=40,1.5+(-0.0125*C22),IF(C22>=10,1,IF(C22>=0,0+(0.1*C22))))
            if vgvc >= 80:
                return 0.5
            elif vgvc >= 40:
                return 1.5+(-0.0125*vgvc)
            elif vgvc >= 10:
                return 1
            else:
                return 0.1*vgvc
        case 'overbank': # IF(C22>=80,0.5,IF(C22>=40,1.5+(-0.0125*C22),IF(C22>=10,1,IF(C22>=0,0+(0.1*C22))))
            if vgvc >= 80:
                return 0.5
            elif vgvc >= 40:
                return 1.5+(-0.0125*vgvc)
            elif vgvc >= 10:
                return 1
            else:
                return 0.1*vgvc
        case 'isolated': # IF(C22>=70,0.5,IF(C22>=25,1.27778+(-0.01111*C22),IF(C22>=0,1)))
            if vgvc >= 70:
                return 0.5
            elif vgvc >= 25:
                return 1.27778+(-0.01111*vgvc)
            else:
                return 1
        case 'connected': # IF(C22>=70,0.5,IF(C22>=25,1.27778+(-0.01111*C22),IF(C22>=0,1))
            if vgvc >= 70:
                return 0.5
            elif vgvc >= 25:
                return 1.27778+(-0.01111*vgvc)
            else:
                return 1                             

# if vohor >= 2 return 1 elif vohor >= 0 return 0.5+(0.25*vohor)
def calc_vohor(vohor: float, subclass: str) -> float:
    match subclass:
        case 'flats': # IF(C23>=2,1,IF(C23>=0,0.5+(0.25*C23))
            if vohor >= 2:
                return 1
            elif vohor >= 0:
                return 0.5+(0.25*vohor)
            else:
                return 0
        case 'backwater':# IF(C23>=2,1,IF(C23>=0,0.5+(0.25*C23))                        
            if vohor >= 2:
                return 1
            elif vohor >= 0:
                return 0.5+(0.25*vohor)
            else:
                return 0
        case 'overbank':# IF(C23>=1,1,IF(C23>=0,0.5+(0.5*C23))                      
            if vohor >= 2:
                return 1
            elif vohor >= 0:
                return 0.5+(0.5*vohor)
            else:
                return 0
        case _:
            return 0
                            
# if vahor >= 8 return 0.5 elif vahor >= 5 return 1.833333+(-0.16667*vahor) elif vahor >= 2 return 1 else return 0.5+(0.25*vahor)
def calc_vahor(vahor: float, subclass: str) -> float:
    match subclass:
        case 'flats': # IF(C24>=8,0.5,IF(C24>=5,1.833333+(-0.16667*C24),IF(C24>=2,1,IF(C24>=0,0.5+(0.25*C24))))                           
            if vahor >= 8:
                return 0.5
            elif vahor >= 5:
                return 1.833333+(-0.16667*vahor)
            elif vahor >= 2:
                return 1
            else:
                return 0.5+(0.25*vahor)
        case 'backwater': # IF(C24>=8,0.5,IF(C24>=5,1.833333+(-0.16667*C24),IF(C24>=2,1,IF(C24>=0,0.5+(0.25*C24))))                       
            if vahor >= 8:
                return 0.5
            elif vahor >= 5:
                return 1.833333+(-0.16667*vahor)
            elif vahor >= 2:
                return 1
            else:
                return 0.5+(0.25*vahor)
        case 'overbank': # IF(C24>=7,0.5,IF(C24>=4,1.666667+(-0.1667*C24),IF(C24>=2,1,IF(C24>=0,0.5+(0.25*C24))))
            if vahor >= 7:
                return 0.5
            elif vahor >= 4:
                return 1.666667+(-0.1667*vahor)
            elif vahor >= 2:
                return 1
            else:
                return 0.5+(0.25*vahor)
        case _:
            return 0

In [35]:
def calcfci(WAAacreage, vtract, vcore, vconnect, vfreq, rnd=0):
    years = [0,5,10,20,35,50]

    targetyearfci = {0:{
        'vtract' : vtract,
        'vcore' : vcore,
        'vconnect' : vconnect,
        'vfreq' : vfreq,
        'vpond' : 45,
        'vsoil' : 50,
        'vcec' : 50,
        'vtba' : 0,
        'vtden' : 0,
        'vsnag' : 0,
        'vtcomp' : 0,
        'vcomp' : 0,
        'vwd' : 0,
        'vlog' : 0,
        'vssd' : 0,
        'vgvc' : 0,
        'vohor' : 5,
        'vahor' : 5},
        5:{
        'vtract' : vtract,
        'vcore' : vcore,
        'vconnect' : vconnect,
        'vfreq' : vfreq,
        'vpond' : 45,
        'vsoil' : 50,
        'vcec' : 50,
        'vtba' : 0,
        'vtden' : 0,
        'vsnag' : 0,
        'vtcomp' : 89,
        'vcomp' : 89,
        'vwd' : 0,
        'vlog' : 0,
        'vssd' : 538,
        'vgvc' : 65,
        'vohor' : 0.46,
        'vahor' : 1},
        10:{
        'vtract' : vtract,
        'vcore' : vcore,
        'vconnect' : vconnect,
        'vfreq' : vfreq,
        'vpond' : 45,
        'vsoil' : 50,
        'vcec' : 50,
        'vtba' : 3,
        'vtden' : 147,
        'vsnag' : 1,
        'vtcomp' : 93,
        'vcomp' : 93,
        'vwd' : 6,
        'vlog' : 5,
        'vssd' : 966,
        'vgvc' : 65,
        'vohor' : 0.38,
        'vahor' : 1},
        20:{
        'vtract' : vtract,
        'vcore' : vcore,
        'vconnect' : vconnect,
        'vfreq' : vfreq,
        'vpond' : 45,
        'vsoil' : 0,
        'vcec' : 0,
        'vtba' : 10,
        'vtden' : 344,
        'vsnag' : 1,
        'vtcomp' : 87,
        'vcomp' : 87,
        'vwd' : 27,
        'vlog' : 17,
        'vssd' : 727,
        'vgvc' : 51,
        'vohor' : 0.56,
        'vahor' : 1},
        35:{
        'vtract' : vtract,
        'vcore' : vcore,
        'vconnect' : vconnect,
        'vfreq' : vfreq,
        'vpond' : 45,
        'vsoil' : 0,
        'vcec' : 0,
        'vtba' : 25,
        'vtden' : 725,
        'vsnag' : 33,
        'vtcomp' : 93,
        'vcomp' : 93,
        'vwd' : 38,
        'vlog' : 29,
        'vssd' : 4000,
        'vgvc' : 43,
        'vohor' : 2,
        'vahor' : 3},
        50:{
        'vtract' : vtract,
        'vcore' : vcore,
        'vconnect' : vconnect,
        'vfreq' : vfreq,
        'vpond' : 45,
        'vsoil' : 0,
        'vcec' : 0,
        'vtba' : 30,
        'vtden' : 650,
        'vsnag' : 28,
        'vtcomp' : 93,
        'vcomp' : 93,
        'vwd' : 48,
        'vlog' : 40,
        'vssd' : 2500,
        'vgvc' : 30,
        'vohor' : 2,
        'vahor' : 3}              
        }
    # Create Subindex
    allsubindices={}

    for a in ['flats', 'backwater', 'overbank', 'isolated', 'connected']:
        tracksub = {}
        for yr in range(len(years)):
            year = years[yr]
            subindex = {
            'vtract' : calc_vtract(targetyearfci[year]['vtract'], a),
            'vcore' : calc_vcore(targetyearfci[year]['vcore'], a),
            'vconnect' : calc_vconnect(targetyearfci[year]['vconnect'], a),
            'vfreq' : calc_vfreq(targetyearfci[year]['vfreq'], a),
            'vpond' : calc_vpond(targetyearfci[year]['vpond'], a),
            'vsoil' : calc_vsoil(targetyearfci[year]['vsoil'], a),
            'vcec' : calc_vcec(targetyearfci[year]['vcec'], a),
            'vtba' : calc_vtba(targetyearfci[year]['vtba'], a),
            'vtden' : calc_vtden(targetyearfci[year]['vtden'], a),
            'vsnag' : calc_vsnag(targetyearfci[year]['vsnag'], a),
            'vtcomp' : calc_vtcomp(targetyearfci[year]['vtcomp'], a),
            'vcomp' : calc_vcomp(targetyearfci[year]['vcomp'], a),
            'vwd' : calc_vwd(targetyearfci[year]['vwd'], a),
            'vlog' : calc_vlog(targetyearfci[year]['vlog'], a),
            'vssd' : calc_vssd(targetyearfci[year]['vssd'], a),
            'vgvc' : calc_vgvc(targetyearfci[year]['vgvc'], a),
            'vohor' : calc_vohor(targetyearfci[year]['vohor'], a),
            'vahor' : calc_vahor(targetyearfci[year]['vahor'], a)
            }                
            tracksub[year]=subindex
        allsubindices[a]=tracksub

    # Single calculation
    trackyearaccs = {}
    # Create Subindex
    for k,v in allsubindices.items():
        trackallaccs = {}
        #if k != 'backwater':
        #    continue
        for yr in range(len(years)):
            year = years[yr]
            v = allsubindices[k][year]
            subacc = {}
            match k:
                case 'flats':
                    subacc = {
                        'dpfci' : (v['vpond']+v['vohor'])/2,
                        'cnfci': (((v['vtba'] + v['vssd'] + v['vgvc'])/3)+((v['vohor'] + v['vahor'] + v['vwd'] + v['vsnag'])/4))/2,
                        'mpcfci' : ((((v['vtba'] + v['vtden'])/2)+v['vcomp'])/2)*((v['vsoil'] + v['vpond'])/2)**(1/2),
                        'pfwhfci' : ((v['vpond'])*((v['vtcomp'] + v['vsnag'] + v['vtba'])/3)*((v['vlog'] + v['vohor'])/2)*((v['vtract'] + v['vconnect'] + v['vcore'])/3))**(1/4)
                    }
                case 'backwater':
                    subacc = {
                        'dffci' : v['vfreq']*((v['vlog']+v['vgvc']+v['vssd']+v['vtden'])/4),
                        'dpfci' : (v['vpond']+v['vohor'])/2,
                        'cnfci': (((v['vtba']+v['vssd']+v['vgvc'])/3)+((v['vohor']+v['vahor']+v['vwd']+v['vsnag'])/4))/2,
                        'eocfci' : v['vfreq']*((((v['vohor']+v['vwd']+v['vsnag'])/3)+((v['vtba']+v['vssd']+v['vgvc'])/3))/2),
                        'recfci' : v['vfreq']*((v['vcec']+v['vohor']+v['vahor'])/3),
                        'mpcfci' : (((((v['vtba']+v['vtden'])/2)+v['vcomp'])/2)*(v['vsoil']+v['vpond'])/2)**(1/2),
                        'pfwhfci' : (((v['vfreq']+v['vpond'])/2)*((v['vtcomp']+v['vsnag']+v['vtba'])/3)*((v['vlog']+v['vohor'])/2)*((v['vtract']+v['vconnect']+v['vcore'])/3))**(1/4)
                    }
                case 'overbank':
                    subacc = {
                        'dffci' : v['vfreq']*((v['vlog']+v['vgvc']+v['vssd']+v['vtden'])/4),
                        'dpfci' : (v['vpond']+v['vohor'])/2,
                        'cnfci': (((v['vtba']+v['vssd']+v['vgvc'])/3)+((v['vohor']+v['vahor']+v['vwd']+v['vsnag'])/4))/2,
                        'eocfci' : v['vfreq']*((((v['vohor']+v['vwd']+v['vsnag'])/3)+((v['vtba']+v['vssd']+v['vgvc'])/3))/2),
                        'recfci' : v['vfreq']*((v['vcec']+v['vohor']+v['vahor'])/3),
                        'mpcfci' : (((((v['vtba']+v['vtden'])/2)+v['vcomp'])/2)*(v['vsoil']+v['vpond'])/2)**(1/2),
                        'pfwhfci' : (((v['vfreq']+v['vpond'])/2)*((v['vtcomp']+v['vsnag']+v['vtba'])/3)*((v['vlog']+v['vohor'])/2)*((v['vtract']+v['vconnect']+v['vcore'])/3))**(1/4)
                    }
                case 'isolated':
                    subacc = {
                        'cnfci': ((v['vtba']+v['vssd']+v['vsnag'])/3),
                        'mpcfci' : (((((v['vtba']+v['vtden'])/2)+v['vcomp'])/2)*v['vsoil'])**(1/2),
                        'pfwhfci' : (((v['vtcomp']+v['vsnag']+v['vtba'])/3)*((v['vtract']+v['vconnect']+v['vcore'])/3))**(1/2)
                    }
                case 'connected':
                    subacc = {
                        'dffci' : v['vfreq']*((v['vssd']+v['vtden'])/2),
                        'cnfci': (v['vtba']+v['vssd']+v['vsnag'])/3,
                        'eocfci' : v['vfreq']*((v['vtba']+v['vssd']+v['vsnag'])/3),
                        'recfci' : v['vfreq']*v['vcec'],
                        'mpcfci' : (((((v['vtba']+v['vtden'])/2)+v['vcomp'])/2)*v['vsoil'])**(1/2),
                        'pfwhfci' : (v['vfreq']*((v['vtcomp']+v['vsnag']+v['vtba'])/3)*((v['vtract']+v['vcore']+v['vconnect'])/3))**(1/3)
                    }
            if rnd:
                for ky,vl in subacc.items():
                    subacc[ky] = round(subacc[ky],1)
            trackallaccs[year]=subacc
        
        trackyearaccs[k]=trackallaccs

    # Calculate AAFCU for each subclass
    functions = ['dffci', 'cnfci', 'dpfci','eocfci', 'recfci', 'mpcfci', 'pfwhfci']
    holdfcu={}
    for k,v in trackyearaccs.items():    
        holdfcutemp={}
        for f in functions:
            fcu = []
            for yr in range(len(years)):
                year = years[yr]
                try:
                    curfci = trackyearaccs[k][year][f]
                except:
                    curfci = 0
                if years[yr] == 0:
                    continue
                else:
                    try:
                        prevyear = years[yr-1]
                        prevfci =  trackyearaccs[k][prevyear][f]
                        newfcu = round((year-years[yr-1])*(((prevfci+curfci)/3) + ((prevfci+curfci)/6)),2)
                    except:
                        newfcu = 0
                    fcu.append(newfcu)
            holdfcutemp[f]=sum(fcu)/50
        holdfcu[k]=holdfcutemp
        
    df = pd.DataFrame.from_dict(holdfcu)
    df = df.rename(index={'dffci':'Detain Floodwater', 'cnfci':'Cycle Nutrients', 'dpfci':'Detain Precipitation', 'eocfci':'Export Organic Carbon','recfci':'Remove Elements and Compounds','mpcfci':'Maintain Plant Communites','pfwhfci':'Provide Fish and Wildlife Habitat'})
    #df = df.rename(columns={'flats':'Flats','backwater':'Riverine Backwater','overbank':'Riverine Overbank','isolated':'Isolated Depressions','connected':'Connected Depressions'})
    df.loc["Total AAFCU per acre"] = df.sum()
    df.loc["Total AAFCUs per WAA"] = WAAacreage*df.loc["Total AAFCU per acre"]
    return df, holdfcu


In [40]:
holdarr=[]
savearr=[]
dt = datetime.datetime.now().strftime('%m_%d_%Y')
for i in range(len(storeflood)):
    indftract = outfoi
    indffreq = storeflood[i]
    indf = indftract.merge(indffreq, on='Name')
    indf['geometry'] = indf['geometry_y']
    hold = pd.DataFrame(columns=['Name', 'index', 'flats', 'backwater', 'overbank', 'isolated', 'connected'])
    savesum = pd.DataFrame(columns=['Name', 'vegtype','Total'])
    for l in range(len(indf)):
        WAAacreage = indf.loc[l]['vegacres']
        vtract = indf.loc[l]['vtract']
        vcore = indf.loc[l]['vcore']
        vconnect = indf.loc[l]['vconnect']
        vfreq = indf.loc[l]['vfreq']
        out, holdstuff = calcfci(WAAacreage,  vtract, vcore, vconnect, vfreq, 1)
        out = out.reset_index()
        out['Name'] = indf.loc[l]['Name']
        out['floodacres'] = indf.loc[l]['floodacres']
        out['vegacres'] = indf.loc[l]['vegacres']
        out = out[['Name','index',indf.loc[l]['vegtype']]]
        savesum = pd.concat([savesum,pd.DataFrame({'Name':indf.loc[l]['Name'],'vegtype':indf.loc[l]['vegtype'],'Total':out.loc[out['index']=='Total AAFCU per acre',indf.loc[l]['vegtype']]*indf.loc[l]['vegacres']})])
        if indf.loc[l]['Name'] in hold['Name'].values:
            hold.loc[(hold['Name'] == indf.loc[l]['Name']),indf.loc[l]['vegtype']] = out[indf.loc[l]['vegtype']]
        else:
            hold = pd.concat([hold, out])
    hold = hold.rename(columns={'index':'Function','flats':'Flats','backwater':'Riverine Backwater','overbank':'Riverine Overbank','isolated':'Isolated Depressions','connected':'Connected Depressions'})
    hold = hold.fillna(0)
    pth = Path(floodfiles[i]).stem
    hold.to_csv(os.path.join(workspace,savefile+'_{0}_{1}.csv'.format(pth, dt)))
    ingdf = gpd.GeoDataFrame(indf)
    ingdf.drop(columns=['geometry_x', 'geometry_y']).to_parquet(os.path.join(workspace,savefile+'_{0}_{1}.parquet'.format(pth, dt)))
    ingdf.drop(columns=['geometry_x', 'geometry_y']).to_file(os.path.join(workspace,savefile+'_{0}_{1}.shp'.format(pth, dt)))
    holdarr.append(hold)
    savearr.append(savesum)
'''
print('--------------------')
print(years[yr])
print(f)
print(k)
print(v)
print(newfcu)
print('--------------------')
'''

C:\Users\mmitchell.DUCKS\AppData\Local\Temp\ipykernel_29852\4130389782.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  savesum = pd.concat([savesum,pd.DataFrame({'Name':indf.loc[l]['Name'],'vegtype':indf.loc[l]['vegtype'],'Total':out.loc[out['index']=='Total AAFCU per acre',indf.loc[l]['vegtype']]*indf.loc[l]['vegacres']})])
C:\Users\mmitchell.DUCKS\AppData\Local\Temp\ipykernel_29852\4130389782.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hold = pd.concat([hold, out])
C:\Users\mmitc

"\nprint('--------------------')\nprint(years[yr])\nprint(f)\nprint(k)\nprint(v)\nprint(newfcu)\nprint('--------------------')\n"

In [56]:
pd.options.display.float_format = "{:,.2f}".format
for i in range(len(storeflood)):
    display(holdarr[i].sort_values(by='Name'))

,Name,Function,Flats,Riverine Backwater,Riverine Overbank,Isolated Depressions,Connected Depressions
8,HGM example 1,Total AAFCUs per WAA,290.680,"4,782.614",0,12.986,0
6,HGM example 1,Provide Fish and Wildlife Habitat,0.735,0.765,0,0.740,0
5,HGM example 1,Maintain Plant Communites,0.705,0.795,0,0.815,0
4,HGM example 1,Remove Elements and Compounds,0.000,0.840,0,0.000,0
3,HGM example 1,Export Organic Carbon,0.000,0.700,0,0.000,0
2,HGM example 1,Detain Precipitation,0.900,0.900,0,0.000,0
1,HGM example 1,Cycle Nutrients,0.665,0.715,0,0.600,0
0,HGM example 1,Detain Floodwater,0.000,0.710,0,0.000,0
7,HGM example 1,Total AAFCU per acre,3.005,5.425,0,2.155,0
2,HGM example 2,Detain Precipitation,0.000,0.900,0,0.000,0


,Name,Function,Flats,Riverine Backwater,Riverine Overbank,Isolated Depressions,Connected Depressions
8,HGM example 1,Total AAFCUs per WAA,290.680,"3,742.340",0.000,12.986,0
6,HGM example 1,Provide Fish and Wildlife Habitat,0.735,0.720,0.000,0.740,0
5,HGM example 1,Maintain Plant Communites,0.705,0.795,0.000,0.815,0
4,HGM example 1,Remove Elements and Compounds,0.000,0.420,0.000,0.000,0
3,HGM example 1,Export Organic Carbon,0.000,0.345,0.000,0.000,0
2,HGM example 1,Detain Precipitation,0.900,0.900,0.000,0.000,0
1,HGM example 1,Cycle Nutrients,0.665,0.715,0.000,0.600,0
0,HGM example 1,Detain Floodwater,0.000,0.350,0.000,0.000,0
7,HGM example 1,Total AAFCU per acre,3.005,4.245,0.000,2.155,0
2,HGM example 2,Detain Precipitation,0.900,0.900,0.000,0.000,0


In [38]:
pd.options.display.float_format = "{:,.1f}".format
for i in range(len(storeflood)):
    if i == 0:
        print('Flood 2')
    else:
        print('Flood 5')
    display(savearr[i][['Name','Total']].groupby('Name').agg('sum'))

Flood 2


,Total
Name,
HGM example 1,"5,086.3"
HGM example 2,"4,364.8"
HGM example 3,"1,873.5"
HGM example 4,191.2
HGM example 5,"6,868.5"


Flood 5


,Total
Name,
HGM example 1,"4,046.0"
HGM example 2,"4,184.8"
HGM example 3,"1,775.6"
HGM example 4,"2,161.6"
HGM example 5,"5,706.9"


In [19]:
#saveorig = savearr
#for i in range(len(storeflood)):
#    display(saveorig[i]['Total'] - savearr[i]['Total'])